In [ ]:
# tsv = [sample key, image url, class name, verification label, h-dimensional feature delimited by ','].

In [1]:
# emb_file = feat_dir + 'extracted_emb.txt'
# img_path_file = feat_dir + 'extracted_imgpaths.txt'

# save_train_tsv_file = feat_dir + 'cleannet_train.tsv'
# save_val_tsv_file = feat_dir + 'cleannet_val.tsv'
# save_all_tsv_file = feat_dir + 'cleannet_all.tsv'
# save_class_names_file = feat_dir + 'cleannet_classnames.txt'
# np.savetxt(save_class_names_file, tsv['class_name'].unique(), fmt='%s')

import numpy as np
import pandas as pd
print("ok")

ok


# Waseem

In [27]:
# sample_key	image_url	class_name	verification_label	feature

train = pd.read_csv('../datasets/waseem/waseem_train.csv', sep='\t')
train['sample_key'] = np.arange(len(train))
print(train.head())

# t.to_csv(save_tsv_file, sep='\t', index=False, header=None)
#         print("tsv created", t.head())
print(len(train))

                   id                                               text  \
0  597576902212063232  Cisco had to deal with a fat cash payout to th...   
1  595380689534656512  guys. show me the data. show me your github. t...   
2  563757610327748608  @tpw_rules nothings broken. I was just driving...   
3  563082741370339330     ur face is classified as a utility by the FCC.   
4  596962098845851648  @lysandraws yay! Absolutely. I'm not gone unti...   

   gt_label  label  vlabel  test  sample_key  
0         3      3       1     0           0  
1         3      3       1     0           1  
2         3      3       1     0           2  
3         3      3       1     0           3  
4         3      3       1     0           4  
5527


In [52]:
# sample_key	image_url	class_name	verification_label	feature

train_emb = np.loadtxt('/home/krsharma/labelnoise/trained_models/waseem/emb_epoch.txt')
print(train_emb.shape)

(5527, 768)


In [40]:
from sklearn.model_selection import train_test_split

ind = np.arange(len(train))
ind_train, ind_test, gt_train, gt_test = train_test_split(ind, np.array(train.gt_label), test_size=0.15, random_state=42, stratify=train.gt_label)
print(len(ind_train))
print(len(ind_test))
print(len(gt_train))
print(len(gt_test))



4697
830
4697
830


In [46]:
allsamples = pd.DataFrame()

allsamples['sample_key'] = train['sample_key']
allsamples['image_url'] = train['id']
allsamples['class_name'] = train['label']
allsamples['verification_label'] = train['vlabel']
allsamples['feature'] = [",".join(map(str, emb)) for emb in train_emb]

In [43]:

vtrain = allsamples.iloc[ind_test] # 15% = 830 for verified train, rest can be used for verified validation.
vval = allsamples.iloc[ind_train]
vtrain.head()

,sample_key,image_url,class_name,verification_label,feature
2779,2779,571261793567436800,3,1,"-0.431,-0.017,0.393,0.203,0.199,0.081,0.71,0.3..."
2074,2074,571056355580055553,1,1,"-0.337,-0.182,-0.771,-0.247,-0.021,0.125,-0.43..."
4431,4431,576097413275897856,3,1,"-0.523,-0.19,0.146,0.317,0.746,-0.626,0.396,0...."
2613,2613,576089368395304960,3,1,"-0.697,-0.278,0.253,0.302,0.317,-0.28,0.319,0...."
3536,3536,603671235009126401,3,1,"-0.337,-0.182,-0.771,-0.247,-0.021,0.125,-0.43..."


In [44]:
vval.head()

,sample_key,image_url,class_name,verification_label,feature
818,818,595376073983983616,3,1,"0.513,-0.061,0.267,0.194,0.613,0.062,0.3,0.252..."
4694,4694,572200958643580928,1,1,"-0.337,-0.182,-0.771,-0.247,-0.021,0.125,-0.43..."
1357,1357,576190185085374464,3,1,"-0.039,-0.203,0.081,0.163,-0.004,0.404,0.889,0..."
1123,1123,599062574798213120,3,1,"-0.068,-0.271,-0.101,0.566,0.349,-0.303,0.221,..."
254,254,597135449086296064,3,1,"-0.267,0.107,0.263,-0.072,0.049,0.278,0.435,0...."


In [45]:
allsamples.head()

,sample_key,image_url,class_name,verification_label,feature
0,0,597576902212063232,3,1,"-0.59,0.027,0.081,0.871,0.133,0.058,0.878,0.39..."
1,1,595380689534656512,3,1,"-0.266,-0.159,0.117,0.089,0.348,0.061,0.612,0...."
2,2,563757610327748608,3,1,"-0.204,-0.418,0.075,-0.104,0.282,-0.31,0.412,0..."
3,3,563082741370339330,3,1,"-0.936,-0.129,0.476,0.173,0.176,0.25,0.418,-0...."
4,4,596962098845851648,3,1,"-0.674,-0.397,0.341,0.355,0.314,-0.275,0.446,0..."


In [47]:
vtrain.to_csv('../trained_models/waseem/cleannet_train.tsv', sep='\t', index=False, header=None)


In [48]:
vval.to_csv('../trained_models/waseem/cleannet_val.tsv', sep='\t', index=False, header=None)


In [49]:
cols = ['sample_key', 'image_url', 'class_name', 'feature']
allsamples[cols].to_csv('../trained_models/waseem/cleannet_all.tsv', sep='\t', index=False, header=None)


In [51]:
print(len(allsamples), len(vtrain), len(vval))

5527 830 4697


In [3]:
import pandas as pd
noiserank_vval = pd.read_csv('../trained_models/waseem/cleannet_val.tsv', sep='\t', header=None)
noiserank_vval.head()

,0,1,2,3,4
0,818,595376073983983616,3,1,"0.513,-0.061,0.267,0.194,0.613,0.062,0.3,0.252..."
1,4694,572200958643580928,1,1,"-0.337,-0.182,-0.771,-0.247,-0.021,0.125,-0.43..."
2,1357,576190185085374464,3,1,"-0.039,-0.203,0.081,0.163,-0.004,0.404,0.889,0..."
3,1123,599062574798213120,3,1,"-0.068,-0.271,-0.101,0.566,0.349,-0.303,0.221,..."
4,254,597135449086296064,3,1,"-0.267,0.107,0.263,-0.072,0.049,0.278,0.435,0...."


In [5]:
del noiserank_vval[4] # sample_key	image_url	class_name	verification_label	feature

In [6]:
noiserank_vval.head() # [sample key, content url, class name, verification label]

,0,1,2,3
0,818,595376073983983616,3,1
1,4694,572200958643580928,1,1
2,1357,576190185085374464,3,1
3,1123,599062574798213120,3,1
4,254,597135449086296064,3,1


In [7]:
noiserank_vval.to_csv('../trained_models/waseem/noiserank_val.tsv', sep='\t', index=False, header=None)